In [1]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd
from pyswip import Prolog
 
# Διάβασμα του αρχείου 'movie_metadata.csv' 
path = r"C:\Users\OWNER\Desktop\AI Σειρα 2\2ο Θέμα"
data = pd.read_csv(path + "\movies_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 


In [2]:
import pandas as pd
from pyswip import Prolog

def clean_text(text):
  text = text.replace(u'\xa0', u'')
  text = text.replace(u"'", u'')
  return text

#create World
#Ορίζουμε τον κόσμο μας
prolog = Prolog()

#Για κάθε row του πίνακα φτιάχνουμε τα κατηγορήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
 
for row in data.itertuples(index=True, name='Pandas'):
  movie_title = clean_text(getattr(row, 'movie_title'))

#ids epeidh oi titloi Out of the blue, Batman, The host yparxoun parapanw apo 2 fores
  ident = getattr(row, 'id')
  literals.append("identification('"+ movie_title +"','"+ str(ident) +"')")

#plot keywords
  for plotkeys in getattr(row, 'plot_keywords').split("|"):
    plot = clean_text(plotkeys)
    literals.append("plot('"+ str(ident) +"','"+ plot +"')")

#language
  lang = getattr(row, 'language')
  literals.append("language('"+ str(ident) +"','"+ lang +"')")

#genre
  for genre in getattr(row, 'genres').split("|"):
    literals.append("genre('"+ str(ident) +"','"+ genre +"')")

#popularity
  pop = getattr(row, 'popularity')
  literals.append("popularity('"+ str(ident) +"','"+ str(pop) +"')")  # den evala int alla string

#production company
  comp = getattr(row, 'production_companies')
  if comp != '[]':
    arr = comp.replace('[','').replace('}]','').split('}, ')
    for s in arr:
      s = clean_text(s).replace('{name: ','').replace(', id: ',',').split(',')[0]
      literals.append("production_company('"+ str(ident) +"','"+ s +"')")
  else:
    literals.append("production_company('"+ str(ident) +"','""')") #indie

#production country
  countries = getattr(row, 'production_countries')
  if countries != '[]':
    countries = countries.replace('[','').replace('}]','').split('}, ')
    for s in countries:
      s = clean_text(s).replace('name: ','').replace(', id: ',',').split(',')[1]
      literals.append("production_country('"+ str(ident) +"','"+ s +"')")
  else:
    literals.append("production_country('"+ str(ident) +"','""')")

#release date
  date = getattr(row, 'release_date')
  literals.append("release_date('"+ str(ident) +"','"+ date +"')")

#duration
  dur = getattr(row, 'duration')
  literals.append("duration('"+ str(ident) +"','"+ str(dur) +"')")

#languages spoken
  slang = getattr(row, 'spoken_languages')
  if slang != '[]':
    slang = slang.replace('[','').replace('}]','').split('}, ')
    for s in slang:
      s = clean_text(s).replace('name: ','').replace(', id: ',',').split(',')[1]
      literals.append("spoken_languages('"+ str(ident) +"','"+ s +"')")
  else:
    literals.append("spoken_languages('"+ str(ident) +"','""')")

#reviews
  avg = getattr(row, 'vote_average')
  literals.append("vote_average('"+ str(ident) +"','"+ str(avg) +"')")

#year
  year = getattr(row, 'title_year')
  literals.append("title_year('"+ str(ident) +"','"+ str(year) +"')")

#director
  director = getattr(row, 'director_name')
  literals.append("director('"+ str(ident) +"','"+ clean_text(director) +"')")

#actor1
  actor1 = getattr(row, 'actor_1_name')
  literals.append("actor_1('"+ str(ident) +"','"+ clean_text(actor1) +"')")

#actor2
  actor2 = getattr(row, 'actor_2_name')
  literals.append("actor_2('"+ str(ident) +"','"+ clean_text(actor2) +"')")

#actor3
  actor3 = getattr(row, 'actor_3_name')
  literals.append("actor_3('"+ str(ident) +"','"+ clean_text(actor3) +"')")
    
#Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
  prolog.assertz(literal)
  #print (literal +'.')

In [3]:
import pandas as pd
from pyswip import Prolog

prolog = Prolog()
changeme = "/Users/OWNER/Desktop/testdrive.pl"
prolog.consult(changeme)

In [22]:
def s_recommender(movie):
    q1 = prolog.query("esp_vg('" + movie +"',L)")
    esp_vg = next(q1)
    esp_vg = {variable: [str(elem) for elem in esp_vg[variable]] for variable in esp_vg}
    q1.close()
    
    q2 = prolog.query("esp_sg('" + movie +"',L)")
    esp_sg = next(q2)
    esp_sg = {variable: [str(elem) for elem in esp_sg[variable]] for variable in esp_sg}    
    q2.close()
    
    q3 = prolog.query("esp_pg('" + movie +"',L)")
    esp_pg = next(q3)
    esp_pg = {variable: [str(elem) for elem in esp_pg[variable]] for variable in esp_pg}    
    q3.close()
    
    q4 = prolog.query("es_plot('" + movie +"',L)")
    esp = next(q4)
    esp = {variable: [str(elem) for elem in esp[variable]] for variable in esp}    
    q4.close()
    
    L1 = esp_vg['L']
    L2 = esp_sg['L']
    L3 = esp_pg['L']
    L4 = esp['L']
    
    for movie in L4:
        if movie in L1 or movie in L2 or movie in L3:
            L4.remove(movie)
            
    answer = L1 + L2 + L3 + L4
     
    return answer

def a_recommender(movie):
    q1 = prolog.query("vsp_vg('" + movie +"',L)")
    vsp_vg = next(q1)
    vsp_vg = {variable: [str(elem) for elem in vsp_vg[variable]] for variable in vsp_vg}
    q1.close()
    
    q2 = prolog.query("vsp_sg('" + movie +"',L)")
    vsp_sg = next(q2)
    vsp_sg = {variable: [str(elem) for elem in vsp_sg[variable]] for variable in vsp_sg}    
    q2.close()
    
    q3 = prolog.query("vsp_pg('" + movie +"',L)")
    vsp_pg = next(q3)
    vsp_pg = {variable: [str(elem) for elem in vsp_pg[variable]] for variable in vsp_pg}    
    q3.close()
    
    q4 = prolog.query("vs_plot('" + movie +"',L)")
    vsp = next(q4)
    vsp = {variable: [str(elem) for elem in vsp[variable]] for variable in vsp}    
    q4.close()
    
    L1 = vsp_vg['L']
    L2 = vsp_sg['L']
    L3 = vsp_pg['L']
    L4 = vsp['L']
    
    for movie in L4:
        if movie in L1 or movie in L2 or movie in L3:
            L4.remove(movie)
    
    answer = L1 + L2 + L3 + L4
     
    return answer

def b_recommender(movie):
    q1 = prolog.query("sp_vg('" + movie +"',L)")
    sp_vg = next(q1)
    sp_vg = {variable: [str(elem) for elem in sp_vg[variable]] for variable in sp_vg}
    q1.close()
    
    q2 = prolog.query("sp_sg('" + movie +"',L)")
    sp_sg = next(q2)
    sp_sg = {variable: [str(elem) for elem in sp_sg[variable]] for variable in sp_sg}    
    q2.close()
    
    q3 = prolog.query("sp_pg('" + movie +"',L)")
    sp_pg = next(q3)
    sp_pg = {variable: [str(elem) for elem in sp_pg[variable]] for variable in sp_pg}    
    q3.close()
    
    q4 = prolog.query("s_plot('" + movie +"',L)")
    sp = next(q4)
    sp = {variable: [str(elem) for elem in sp[variable]] for variable in sp}    
    q4.close()
    
    q5 = prolog.query("vs_genre('" + movie +"',L)")
    vsg = next(q5)
    vsg = {variable: [str(elem) for elem in vsg[variable]] for variable in vsg}    
    q5.close()
    
    L1 = sp_vg['L']
    L2 = sp_sg['L']
    L3 = sp_pg['L']
    L4 = sp['L']
    L5 = vsg['L']
    
    for movie in L4:
        if movie in L1 or movie in L2 or movie in L3:
            L4.remove(movie)
    
    answer = L1 + L2 + L3 + L4
     
    return answer

In [23]:
s_recommender('Harry Potter and the Half-Blood Prince')

['Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Order of the Phoenix',
 'Harry Potter and the Philosophers Stone',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Philosophers Stone']

In [24]:
a_recommender('Harry Potter and the Half-Blood Prince')

['Oz: The Great and Powerful',
 'Spirited Away',
 'Conan the Destroyer',
 'Hotel Transylvania',
 'In the Name of the King: A Dungeon Siege Tale',
 'The Sorcerers Apprentice',
 'Warlock',
 'Goosebumps',
 'Hotel Transylvania 2',
 'Practical Magic',
 'The Craft',
 'The Huntsman: Winters War',
 'Goosebumps',
 'Hotel Transylvania',
 'Oz: The Great and Powerful',
 'Spirited Away',
 'The Huntsman: Winters War',
 'Warlock']

In [25]:
b_recommender('Harry Potter and the Half-Blood Prince')

['A Simple Wish',
 'Aladdin',
 'Asterix at the Olympic Games',
 'Last Action Hero',
 'Pan',
 'Shrek 2',
 'Shrek',
 'Stardust',
 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe',
 'The Chronicles of Narnia: The Voyage of the Dawn Treader',
 'The NeverEnding Story',
 'The Smurfs',
 'The Thief and the Cobbler',
 'The Wizard of Oz',
 'Thunder and the House of Magic',
 'Time Bandits',
 'Big Fish',
 'Big Trouble in Little China',
 'Bogus',
 'Cinderella',
 'Ella Enchanted',
 'Happily NEver After',
 'Hocus Pocus',
 'Howls Moving Castle',
 'Mary Poppins',
 'Nanny McPhee',
 'Pirates of the Caribbean: Dead Mans Chest',
 'Seventh Son',
 'Snow White and the Huntsman',
 'Snow White and the Seven Dwarfs',
 'Teen Wolf Too',
 'The 13th Warrior',
 'The Covenant',
 'The Haunted Mansion',
 'The Last Witch Hunter',
 'The Mortal Instruments: City of Bones',
 'The Santa Clause 2',
 'The Twilight Saga: Eclipse',
 'The Twilight Saga: New Moon',
 'Underworld: Rise of the Lycans',
 'Warcraft',
 '

In [30]:
from tqdm.notebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import random

rating_weights = {0: -1, 1: -0.5, 2:0, 3:0, 4:0.5, 5:1}
score_weights = {i:i + 1 for i in range(3)} # ανάλογα με τα επίπεδα ομοιότητας που έχουν οριστεί στην simple_recommender

def train_recommender(ratings, rating_weights, score_weights, number_of_movies = 10):
    """
    Στην συνάρτηση αυτή μπορούμε να ορίζουμε ποιο υποσύνολο των αξιολογήσεων θα χρησιμοποιήσουμε για το train μαζί με τα βάρη ομοιότητας και σκορ
    Σε συνδυασμό με τον αριθμό των ταινιών που θέλουμε να χρησιμοποιήσουμε σαν σύνολο δεδομένων π.χ. 10 από τις 100 ή 3 από τις 100 κ.ο.κ
    Αν θέλουμε να χρησιμοποιήσουμε όλες τις ταινίες σαν training set τότε ορίζουμε το number_of_movies = - 1
    """

    if number_of_movies > len(ratings):
        number_of_movies = len(ratings)


    if number_of_movies != -1:
        indexes = random.sample(range(len(ratings)), number_of_movies)
        ratings = ratings.iloc[indexes]

    movie_score = {}
    for row in tqdm(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        ps_movies = s_recommender(movie)
        es_movies = a_recommender(movie)
        vs_movies = b_recommender(movie)
        
        
        for similar_movie in ps_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[2]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[2]
                
        for similar_movie in es_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[1]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[1]
                
        for similar_movie in vs_movies:
            if similar_movie not in movie_score:
                movie_score[similar_movie] = rating_weights[int(rating)] * score_weights[0]
            else:
                movie_score[similar_movie] += rating_weights[int(rating)] * score_weights[0]
        
    return movie_score


#αυτό είναι ένα παράδειγμα για το πως θα μπορούσε να υλοποιήθει η predict
#έχουμε ορίσει ότι μια ταινία θα έπρεπε να είναι προτεινόμενη αν είχε σκορ > 0
def predict_example(ratings, movie_score):
    real, pred = [], []
    for i, row in enumerate(ratings.itertuples(index=True, name='Pandas')):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')

        if movie in movie_score: #αν έχουμε σχηματίσει βαθμολογία για την ταινία αυτή
            pred.append(int(movie_score[movie] > 0)) #heuristic για το αν μια ταινία είναι προτεινόμενη
            real.append(int(rating > 3))# έτσι ορίζουμε ότι μια ταινία θα έπρεπε να είναι προτεινόμενη
            #η συνθήκη αυτή δεν μπορεί να αλλάξει
        else: #δεν μπορούμε να προτείνουμε κάτι για το όποιο δεν έχουμε σχηματίσει εικόνα
            pred.append(0)
            real.append(int(rating > 3))

    return real, pred


def get_metrics(real, pred):
    metrics = {}
    metrics["precision"] = precision_score(real, pred)
    metrics["recall"] = recall_score(real, pred)
    metrics["f1"] = f1_score(real, pred)
    return metrics

In [31]:
train_ratings = pd.read_csv(path + "\\train_ratings.csv")
test_ratings = pd.read_csv(path + "\\test_ratings.csv")
metrics = []
for i in range (10):
    movie_score = train_recommender(train_ratings, rating_weights, score_weights, 10)
    real, pred = predict_example(test_ratings, movie_score)
    metrics.append(get_metrics(real, pred))

for metric in metrics[0].keys():
    print (f"{metric}: {np.mean([m[metric] for m in metrics])}")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

precision: 0.5541414055564999
recall: 0.21527777777777776
f1: 0.28774156546840624
